<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import httpx
from datetime import datetime

In [15]:
# change driver path to your own chrome driver poth
driver_path = r'C:\Users\MLee27\Library\chromedriver_win32\chromedriver.exe'

output = pd.DataFrame(columns=['Index','Shop_Name', 'Prod_Name', 'Currency', 'Price', 'Price_Ori', 'Discount_Rate', 'Unit_Sold',
                               'Product_Description', 'Average_Rating', 'Product_Brand', 'Product_Category', 'URL'])

# paste the url that you wish to scrape
url_input = input('Enter your scrape target url: ')

# key in the name that you want to use for the output csv
file_name = input('Enter your file name: ')

Enter your scrape target url: https://shopee.com.my/tesco.fbs?page=2&shopCollection=21430950&sortBy=pop
Enter your file name: Kitchen_Dining_Scraped


In [16]:
start_time = time.time()


driver = webdriver.Chrome(executable_path = driver_path)
    
all_urls = []

for page in range(1):
        
    driver.get(url_input)

    if page == 0:
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//div[@class="language-selection__list-item"]/button'))).click()
    else:
        WebDriverWait(driver,5)
    time.sleep(3)

    for x in range(10):
        driver.execute_script("window.scrollBy(0,300)")
        time.sleep(0.1)

    all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')

    for item in all_items:
        url = item.get_attribute('href')
        all_urls.append(url)

driver.quit()

index = 0

for url in all_urls:
    ids = url.split('-i.')[1].split('.')
    shop_id = ids[0]
    prod_id = ids[1]

    shop_api = 'https://shopee.com.my/api/v2/shop/get?is_brief=1&shopid=' + str(shop_id)
    shop_name = httpx.get(shop_api).json().get('data')['name']

    item_api = 'https://shopee.com.my/api/v2/item/get?itemid=' +str(prod_id) + '&shopid=' + str(shop_id)
    item_detail = httpx.get(item_api).json().get('item')
    price = item_detail['price_min']/100000
    price_before_discount = item_detail['price_min_before_discount']/100000
    discount_rate = item_detail['discount']
    prod_brand = item_detail['brand']
    prod_categories = item_detail['categories'][0]['display_name']
    currency = item_detail['currency']
    unit_sold = item_detail['historical_sold']
    prod_name = item_detail['name']
    prod_desc = item_detail['description'].replace('\n',' ').replace('  ',' ')
    average_rating = item_detail['item_rating']['rating_star']
    
    output = output.append([{'Index':index, 'Shop_Name':shop_name, 'Prod_Name':prod_name, 'Currency':currency, 'Price':price,
                             'Price_Ori':price_before_discount, 'Discount_Rate':discount_rate, 'Unit_Sold':unit_sold,
                             'Product_Description':prod_desc, 'Average_Rating':average_rating, 'Product_Brand':prod_brand,
                             'Product_Category':prod_categories, 'URL':url}],sort=False)
    
    index += 1
    
output = output.set_index(['Index'])
output.to_csv('ShopeeScrape_' + str(file_name) + '_' + str(datetime.now().strftime('%Y%m%d_%H%M%S')) + '.csv')
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 26.533343076705933 seconds ---
